In [ ]:
import pandas as pd
import time
import re
from tqdm import tqdm
import pubchempy as pcp
tqdm.pandas()

# Funções

In [ ]:
def obter_smiles_cas(row, col_cas='CASRN'):
    try:
        cas_number = row[col_cas]
        if pd.isna(cas_number) or not isinstance(cas_number, str):
            return "Valor inválido"
        cas_number = cas_number.strip().replace(" ", "").replace("\n", "").replace("\r", "")
        if not cas_number.replace("-", "").isdigit():
            return "Formato inválido"
        print(f"Buscando SMILES para CAS: '{cas_number}'")
        time.sleep(0.5)
        composto = pcp.get_compounds(cas_number, 'name')
        return composto[0].canonical_smiles if composto else None
    except pcp.PubChemHTTPError as e:
        print(f"Erro HTTP ao buscar SMILES para CAS {cas_number}: {e}")
        return "Erro HTTP"
    except Exception as e:
        print(f"Erro inesperado para CAS {cas_number}: {e}")
        return "Erro desconhecido"

In [ ]:
def corrigir_cas(cas_number):
    if pd.isna(cas_number) or not isinstance(cas_number, str):
        return None
    cas_number = cas_number.strip().replace(" ", "").replace("\n", "").replace("\r", "")
    if cas_number.startswith("-"):
        return None
    if re.match(r"^\d{2,7}-\d{2}-\d$", cas_number):
        return cas_number
    if cas_number.isdigit():
        length = len(cas_number)
        if length > 3:
            formatted_cas = f"{cas_number[:-3]}-{cas_number[-3:-1]}-{cas_number[-1]}"
            if re.match(r"^\d{2,7}-\d{2}-\d$", formatted_cas):
                return formatted_cas
    return None


In [ ]:
# Define uma função para aplicar a lógica desejada
def definir_opiniao(grupo):
    if 'Active' in grupo.values:
        return '+'
    elif 'Inactive' in grupo.values:
        return '-'
    else:
        return '0'

In [ ]:
def obter_nome(row, col_cid='PUBCHEM_CID'):
    try:
        # Busca o composto pelo CID fornecido na coluna especificada
        composto = pcp.Compound.from_cid(row[col_cid])
        # Retorna o nome IUPAC do composto se encontrado, ou None
        return composto.iupac_name if composto else None
    except Exception as e:
        print(f"Erro ao buscar nome para CID {row[col_cid]}: {e}")
        return None

In [ ]:
def obter_cas(row, col_nome='name'):
    try:
        # Busca o composto pelo nome
        composto = pcp.get_compounds(row[col_nome], 'name')
        if composto:
            # Retorna o CAS Number do primeiro composto encontrado, se existir
            propriedades = composto[0].to_dict(properties=['xlogp', 'iupac_name', 'molecular_weight', 'synonyms'])
            cas_numbers = [sinonimo for sinonimo in propriedades['synonyms'] if '-' in sinonimo and sinonimo.replace('-', '').isdigit()]
            return cas_numbers[0] if cas_numbers else None
        else:
            return None
    except Exception as e:
        print(f"Erro ao buscar CAS Number para {row[col_nome]}: {e}")
        return None

# ECHA

In [ ]:
# Carregamento do arquivo CSV
path = '/content/ECHA .xlsx - Genotox data.csv'
df1 = pd.read_csv(path)

# Remoção de colunas desnecessárias
df1.drop(columns=[
    'SUB_ECSUBINVENTENTRYREF', 'SUB_TYPE', 'QUALIFIER',
    'COM_NAME', 'COM_ECSUBINVENTENTRYREF', 'COM_CASNUMBER',
    'COM_STRUCTURESHOWN'
], inplace=True)

# Supondo que você tenha essa função para processar a coluna 'Strain/Sex'
df1 = split_strain_sex(df1, 'Strain/Sex')


In [ ]:
df1['SMILES'] = df1.progress_apply(obter_smiles_cas, axis=1)

In [ ]:
# Aqui copiamos para trabalhar nos inválidos
df2_nan_smiles = df1.copy()
df2_nan_smiles["CAS No."] = df2_nan_smiles["CAS No."].apply(corrigir_cas)
df2_validos = df2_nan_smiles[df2_nan_smiles["CAS No."].notna()]


In [ ]:
df2 = df1.copy()  # supondo que df2 é o dataframe com SMILES de df1, ou pode ser outro

df_validos = df2[
    (~df2['SMILES'].isna()) &
    (df2['SMILES'] != "Formato inválido") &
    (df2['SMILES'] != "Valor inválido")
]


In [ ]:
df_final = pd.concat([df1, df2], ignore_index=True)

# Garantir que SMILES é string
df_final["SMILES"] = df_final["SMILES"].astype(str)

# Remover duplicatas mantendo as com SMILES mais informativos
df_final = df_final.sort_values(by="SMILES", ascending=False).drop_duplicates(
    subset=["Chemical", "CAS No."], keep="first"
)

# Ordenar e resetar o índice
df_final_1 = df_final.sort_values(by="Chemical", ascending=True).reset_index(drop=True)


# CCRIS

In [ ]:
import pandas as pd
import pubchempy as pcp
from tqdm import tqdm

In [ ]:
# Suponha que o seu DataFrame se chama df e possui uma coluna chamada 'nome'
path = '/content/CCRIS dataset.csv'

df2 = pd.read_csv(path)

In [ ]:
df2 = df[['PUBCHEM_EXT_DATASOURCE_SMILES', 'PUBCHEM_CID', 'PUBCHEM_ACTIVITY_OUTCOME']]

In [ ]:
df2 = df2.dropna()

In [ ]:
df2 = df2.groupby(['PUBCHEM_CID', 'PUBCHEM_EXT_DATASOURCE_SMILES'], as_index=False)['PUBCHEM_ACTIVITY_OUTCOME'].apply(definir_opiniao)

In [ ]:
# Convertendo a coluna de float para int
df2['PUBCHEM_CID'] = df2['PUBCHEM_CID'].astype(int)

In [ ]:
# Aplica a função para cada nome no DataFrame e cria uma nova coluna com o SMILES
df2['name'] = df2.progress_apply(obter_nome, axis=1)

In [ ]:
# Ordenar e resetar o índice
df_final_2 = df2.sort_values(by="Chemical", ascending=True).reset_index(drop=True)

# ECVAM


In [ ]:
# Suponha que o seu DataFrame se chama df e possui uma coluna chamada 'nome'
pathA = '/content/ECVAM_Ames_negative_DB.csv'
pathB = '/content/ECVAM_Ames_positives_DB.csv'

df3A = pd.read_excel(pathA)
df3B = pd.read_excel(pathB)

In [ ]:
# Seleciona apenas as colunas desejadas (verifica se existem nas duas tabelas)
colunas_desejadas = ['Chemical', 'CAS No.', 'Ames Overall ']

# Filtra as colunas e concatena os DataFrames
df3 = pd.concat([
    df3A[colunas_desejadas],
    df3B[colunas_desejadas]
], ignore_index=True)

# (Opcional) Remove duplicatas, se quiser evitar repetições
df3 = df3.drop_duplicates()

In [ ]:
# Aplica a função para cada nome no DataFrame e cria uma nova coluna com o SMILES
df3['SMILES'] = df3.progress_apply(obter_smiles, axis=1)

In [ ]:
df_final_3 = df3.copy()

# Genetox

In [ ]:
path = '/content/GENE-TOX dataset.csv'

df4 = pd.read_csv(path)

In [ ]:
df4_ = df4[['PUBCHEM_CID', 'PUBCHEM_EXT_DATASOURCE_SMILES', 'PUBCHEM_ACTIVITY_OUTCOME' ]]

In [ ]:
df4_.isnull().sum()

In [ ]:
# Preenchendo valores NaN de duas colunas específicas
df4_[['PUBCHEM_CID', 'PUBCHEM_EXT_DATASOURCE_SMILES']] = df4_[['PUBCHEM_CID', 'PUBCHEM_EXT_DATASOURCE_SMILES']].fillna('None')

In [ ]:
df4_ = df4_.dropna()

In [ ]:
df4_[['PUBCHEM_CID']] = df4_[['PUBCHEM_CID']].astype(int)

In [ ]:
df4_ = df4_.groupby(['PUBCHEM_CID', 'PUBCHEM_EXT_DATASOURCE_SMILES'], as_index=False)['PUBCHEM_ACTIVITY_OUTCOME'].apply(definir_opiniao)

In [ ]:
df4_ = df4_[df4_['PUBCHEM_ACTIVITY_OUTCOME'] != '0']

In [ ]:
# Aplica a função para cada nome no DataFrame e cria uma nova coluna com o SMILES
df4_['name'] = df4_.progress_apply(obter_nome, axis=1)

In [ ]:
# Aplica a função apenas nas linhas onde 'name' está nulo
df4_.loc[df4_['CAS'].isnull(), 'CAS'] = df4_[df4_['CAS'].isnull()].progress_apply(obter_cas, axis=1)

In [ ]:
df4_['CAS'] = df4_.progress_apply(obter_cas, axis=1)

In [ ]:
df_final_4 = df4_.copy()

# IRIS

In [ ]:
path = '/content/Integrated Risk Information System.xlsx - simple_list_alpha.csv'

df5 = pd.read_csv(path)

In [ ]:
df5_ = df5['Chemical Name', 'CASRN' , 'Critical Effect Systems'].copy()

In [ ]:
# Criar a nova coluna 'results'
df5_['results'] = df5_['Critical Effect Systems'].apply(
    lambda x: 'negative' if pd.isna(x) or str(x).strip().lower() == 'none' else 'positive'
)

In [ ]:
# Aplica a função para cada nome no DataFrame e cria uma nova coluna com o SMILES
df5_['SMILES'] = df5_.progress_apply(obter_smiles, axis=1)

In [ ]:
df_final_5 = df5_.copy()

# Agregando datasets

In [ ]:
# Supondo que seus DataFrames sejam df1, df2, df3 e df4
df_final = pd.concat([df_final_1, df_final_2, df_final_3, df_final_4, df_final_5], axis=0, ignore_index=True)

In [ ]:
df_final.to_excel('/content/drive/MyDrive/DOC/Dados/carcinogenese/Dados_juntos_simplificado.xlsx', index=False)